# Exercise 3: ChatGPT-Powered NLP on Customer Support Tickets
### Objective
Use ChatGPT API to classify, analyze, and extract actionable insights from unstructured customer service ticket data. Compare traditional keyword-based methods with AI-driven approaches.

## Step 1: Load the Dataset
Load customer support tickets into a DataFrame to begin processing.

In [1]:
import pandas as pd

# Load the dataset
# Change the path below to your path
df = pd.read_csv('/content/sample_data/support_tickets.csv')
df.head()

,ticket_id,customer_comment,product,channel
0,1001,The battery life is terrible and the device ov...,Smartphone,Email
1,1002,I ordered the product two weeks ago and still ...,Tablet,Web Form
2,1003,The price dropped right after I bought it. Ver...,Headphones,Chat
3,1004,"Customer service was helpful, but the setup in...",Smartwatch,Phone
4,1005,Love the product! The sound quality is amazing...,Speaker,Email


## Step 2: Keyword-Based Tagging (Baseline)
Apply a simple rule-based function to tag each comment into categories: Billing, Technical, Access, Other.

In [2]:
def categorize_ticket(text):
    text = text.lower()
    if 'bill' in text or 'charge' in text:
        return 'Billing'
    elif 'error' in text or 'bug' in text:
        return 'Technical'
    elif 'login' in text or 'access' in text:
        return 'Access'
    else:
        return 'Other'

df['keyword_category'] = df['customer_comment'].apply(categorize_ticket)
df[['customer_comment', 'keyword_category']].head()

,customer_comment,keyword_category
0,The battery life is terrible and the device ov...,Other
1,I ordered the product two weeks ago and still ...,Other
2,The price dropped right after I bought it. Ver...,Other
3,"Customer service was helpful, but the setup in...",Other
4,Love the product! The sound quality is amazing...,Other


## Step 3: Classification with ChatGPT API
Use OpenAI’s GPT model to classify customer comments, assess urgency, and suggest an action.

In [3]:
import openai
import os

# Set your API key (ensure it's set in your environment)
os.environ["OPENAI_API_KEY"]= "insert your key here"

# Instantiate the OpenAI client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def classify_with_chatgpt(comment):
    messages = [
        {"role": "system", "content": "You are a helpful customer support assistant."},
        {"role": "user", "content": f"Classify this support comment: '{comment}' into one of the categories (Billing, Technical, Access, Other). Also state the urgency level (Low, Medium, High) and what action should be taken."}
    ]
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    return response.choices[0].message.content

# Example
classify_with_chatgpt(df['customer_comment'].iloc[0])

"Category: Technical\nUrgency Level: High\nAction to Be Taken: The support team should immediately reach out to the customer to address the issue. First, they should offer basic troubleshooting steps to remedy the overheating issue. If these steps don't resolve the problem, they may need to consider a hardware issue and arrange a replacement or repair for the device."

## Step 4: Generate Actionable Recommendations
Use ChatGPT to suggest service improvements based on customer feedback.

In [4]:
def get_recommendations(comment):
    messages = [
        {"role": "system", "content": "You are a business analyst generating strategic recommendations based on customer feedback."},
        {"role": "user", "content": f"Based on this customer feedback, what are three specific actions the company could take to improve customer experience? Please explain briefly.\n\nFeedback: {comment}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    return response.choices[0].message.content

# Example
get_recommendations(df['customer_comment'].iloc[0])

"1. Improve Battery Life: The company could invest in Research and Development to improve the battery life of their devices. This could involve exploring different battery technologies or enhancing power management software. Prolonging battery life would not only increase customer satisfaction but also provide a competitive advantage in the market.\n\n2. Enhance Thermo-regulatory Features: Customers are complaining about the overheating of the device which could potentially harm the device's functionality and user experience. The company could look to improve heat dissemination perhaps by using higher quality or differently designed internal components. It could also develop or enhance current cooling systems. Advancements in these areas would reduce overheating and improve device lifespan.\n\n3. Implement Robust Quality Assurance: To ensure these improvements have the desired effect, the company should implement a thorough Quality Assurance (QA) process which closely monitors product 

## Step 5: Executive Summary Generation
Summarize multiple customer comments into a high-level report for business stakeholders.

In [5]:
def create_summary(comments):
    joined_comments = "\n".join(comments)
    messages = [
        {"role": "system", "content": "You are a customer success analyst summarizing feedback trends."},
        {"role": "user", "content": f"Summarize these customer service comments into an executive summary highlighting strengths, weaknesses, and areas for improvement:\n{joined_comments}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    return response.choices[0].message.content

# Example
create_summary(df['customer_comment'].sample(5).tolist())

"Executive Summary:\n\nStrengths:\n- High-quality products: Customers have expressed satisfaction with the sound quality and usability of the product, representing it as an area of strength.\n- Good customer service: Feedback indicates that the customer service team is helpful and meets customers' needs, this is another positive aspect of the operations.\n\nWeaknesses:\n- Post-purchase price reduction: Some customers are disappointed that the price dropped soon after they made their purchase. This highlights a need for an improved pricing strategy to maintain customer satisfaction.\n- Delivery times: There are complaints about long delivery times, demonstrating a potential weakness in the shipping department.\n- Product Faults: Inadequate battery life and device overheating is another notable concern, indicating some apparent faults.\n\nAreas for Improvement:\n- Clear communication: The complexity of setup instructions needs to be addressed. This can be improved by ensuring instruction